In this week's independent project, you will be working as a data scientist working for an electric car-sharing service company. You have been tasked to process stations data to understand electric car usage over time by solving for the following research question;

Research Question

Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.


# DATA PREPARATION

In [74]:
# Importing Libraries we could use in the analysis

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
# Loading the dataset into the environment
Autolib = pd.read_csv('Autolib_dataset (2).csv')
Autolib

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,0,0,nonexistent,Paris,NaN,paris-suffren-2,STATION,"48.857, 2.2917",75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,6,0,0,0,operational,Paris,NaN,paris-raymondlosserand-145,STATION,"48.83126, 2.313088",75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,3,0,2,0,operational,Le Bourget,NaN,lebourget-johnfitzgeraldkennedy-2,STATION,"48.938103, 2.4286035",93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,3,1,0,1,operational,Paris,NaN,paris-eugeneoudine-51,STATION,"48.8250327, 2.3725162",75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,3,0,0,0,nonexistent,Paris,NaN,paris-portedechamperret-6,PARKING,"48.8862632, 2.2874511",75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27


In [76]:
Autolib.describe()

,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Postal code,Slots,year,month,day,hour,minute
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.0,5000.000000,5000.000000,5000.000000
mean,2.333600,2.333600,0.059600,0.123000,0.253400,82634.878400,1.932400,2018.0,4.0,4.941600,11.509200,29.270000
std,2.035274,2.035274,0.246698,0.356506,0.546304,8835.865721,1.905402,0.0,0.0,2.597063,6.893549,17.231741
min,0.000000,0.000000,0.000000,0.000000,0.000000,75001.000000,0.000000,2018.0,4.0,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,75012.000000,0.000000,2018.0,4.0,3.000000,6.000000,14.000000
50%,2.000000,2.000000,0.000000,0.000000,0.000000,75019.000000,1.000000,2018.0,4.0,5.000000,11.000000,29.000000
75%,4.000000,4.000000,0.000000,0.000000,0.000000,92320.000000,3.000000,2018.0,4.0,7.000000,18.000000,44.000000
max,7.000000,7.000000,2.000000,3.000000,2.000000,95880.000000,7.000000,2018.0,4.0,9.000000,23.000000,59.000000


In [77]:
Autolib.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

# DATA CLEANING

## BLUE CAR ONLY

In [78]:
# Checking if the Cars and Blue_Counter columns hold the same values.

Autolib_Check = Autolib.Cars == Autolib['Bluecar counter']
Autolib_Check.describe()
# As expected, they do, so we can drop the Cars column.

count     5000
unique       1
top       True
freq      5000
dtype: object

In [79]:
# Checking the relationship between ID, Public name, Postal code and Address
print(Autolib.ID.describe())
print(Autolib['Public name'].describe())

print(Autolib['Postal code'].value_counts())
Autolib.Address.describe()
# ID corresponds to Public name. They both correspond to Address but there seems to be 1 outlier.
# Postal code seems to be different from the above.

# We'll drop ID and Public Name regardless, as we want to know the Addresses and Postal Codes.

count                         5000
unique                        1105
top       paris-portedemontrouge-8
freq                            13
Name: ID, dtype: object
count                           5000
unique                          1105
top       Paris/Porte de Montrouge/8
freq                              13
Name: Public name, dtype: object
75015    274
75016    236
75017    221
75011    180
75020    169
        ... 
93300      5
92110      4
91400      4
93310      4
94100      2
Name: Postal code, Length: 107, dtype: int64


count                                  5000
unique                                 1104
top       8 Avenue de la Porte de Montrouge
freq                                     13
Name: Address, dtype: object

In [80]:
# Dropping unrequired columns

Bluecar = Autolib.drop(['Cars','Utilib counter','Utilib 1.4 counter','Displayed comment','ID','Kind','Geo point','Public name','Subscription status'], axis=1)
Bluecar.head()

,Address,Bluecar counter,Charge Slots,Charging Status,City,Postal code,Rental status,Scheduled at,Slots,Station type,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,2018,4,8,17,23


In [81]:
Bluecar.columns = Bluecar.columns.str.replace(' ', '_')

In [82]:
# We need to convert the days and time to datetime for easier Analysis
# First let's convert them to objects i.e. strings
print(Bluecar.info())

Bluecar.year = Bluecar.year.astype(str)
Bluecar.month = Bluecar.month.astype(str)
Bluecar.day = Bluecar.day.astype(str)

Bluecar.hour = Bluecar.hour.astype(str)
Bluecar.minute = Bluecar.minute.astype(str)

Bluecar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Address          5000 non-null   object
 1   Bluecar_counter  5000 non-null   int64 
 2   Charge_Slots     5000 non-null   int64 
 3   Charging_Status  5000 non-null   object
 4   City             5000 non-null   object
 5   Postal_code      5000 non-null   int64 
 6   Rental_status    5000 non-null   object
 7   Scheduled_at     47 non-null     object
 8   Slots            5000 non-null   int64 
 9   Station_type     5000 non-null   object
 10  Status           5000 non-null   object
 11  year             5000 non-null   int64 
 12  month            5000 non-null   int64 
 13  day              5000 non-null   int64 
 14  hour             5000 non-null   int64 
 15  minute           5000 non-null   int64 
dtypes: int64(9), object(7)
memory usage: 625.1+ KB
None
<class 'pandas.core.frame.

In [83]:
Bluecar.head()

,Address,Bluecar_counter,Charge_Slots,Charging_Status,City,Postal_code,Rental_status,Scheduled_at,Slots,Station_type,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,2018,4,8,17,23


In [84]:
# Next we concatenate them

Blue = Bluecar['year'] + Bluecar['month'] + Bluecar['day']
print(Blue.head())
Test = pd.to_datetime(Blue, format='%Y%m%d', errors='ignore')
print(Test.head())

Time = Bluecar.hour + Bluecar.minute
print(Time.head())

Time_Test = pd.to_datetime(Time, format= '%H%M').dt.time # option 1 gives dtype as object, which will be hard to work with in calculations
print(Time_Test.head())

Time_Test2 = pd.to_datetime(Time, format= '%H%M')- pd.to_datetime(Time, format='%H%M').dt.normalize()
Time_Test2 # option 2 is better due to the datatype timedelta.

0    201848
1    201846
2    201843
3    201844
4    201848
dtype: object
0   2018-04-08
1   2018-04-06
2   2018-04-03
3   2018-04-04
4   2018-04-08
dtype: datetime64[ns]
0    1143
1     724
2    2014
3     437
4    1723
dtype: object
0    11:43:00
1    07:24:00
2    20:14:00
3    04:37:00
4    17:23:00
dtype: object


0      0 days 11:43:00
1      0 days 07:24:00
2      0 days 20:14:00
3      0 days 04:37:00
4      0 days 17:23:00
             ...      
4995   0 days 11:26:00
4996   0 days 16:56:00
4997   0 days 07:01:00
4998   0 days 17:27:00
4999   0 days 10:02:00
Length: 5000, dtype: timedelta64[ns]

In [85]:
Bluecar.head()

,Address,Bluecar_counter,Charge_Slots,Charging_Status,City,Postal_code,Rental_status,Scheduled_at,Slots,Station_type,Status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,2018,4,8,17,23


In [86]:
# After Analysis, it's better to convert only the Year Month and Days to Date_Time and leave the hours as is.
# Furthermore, since the analysis is on hours, the minute column can be dropped.

Bluecar.hour = Bluecar.hour.astype(int)
Bluecar.minute = Bluecar.minute.astype(int)
Bluecar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Address          5000 non-null   object
 1   Bluecar_counter  5000 non-null   int64 
 2   Charge_Slots     5000 non-null   int64 
 3   Charging_Status  5000 non-null   object
 4   City             5000 non-null   object
 5   Postal_code      5000 non-null   int64 
 6   Rental_status    5000 non-null   object
 7   Scheduled_at     47 non-null     object
 8   Slots            5000 non-null   int64 
 9   Station_type     5000 non-null   object
 10  Status           5000 non-null   object
 11  year             5000 non-null   object
 12  month            5000 non-null   object
 13  day              5000 non-null   object
 14  hour             5000 non-null   int64 
 15  minute           5000 non-null   int64 
dtypes: int64(6), object(10)
memory usage: 625.1+ KB


In [87]:
Bluecar['Date'] = Test
Bluecar.head()

,Address,Bluecar_counter,Charge_Slots,Charging_Status,City,Postal_code,Rental_status,Scheduled_at,Slots,Station_type,Status,year,month,day,hour,minute,Date
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,2018,4,8,11,43,2018-04-08
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,2018,4,6,7,24,2018-04-06
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,2018,4,3,20,14,2018-04-03
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,2018,4,4,4,37,2018-04-04
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,2018,4,8,17,23,2018-04-08


In [88]:
# We can now drop year, month and day columns
# Additionally, since the analysis is on hours, the minute column can be dropped.

Bluecar.drop(['year', 'month', 'day', 'minute'], axis=1, inplace= True)
Bluecar.head()

,Address,Bluecar_counter,Charge_Slots,Charging_Status,City,Postal_code,Rental_status,Scheduled_at,Slots,Station_type,Status,hour,Date
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,11,2018-04-08
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,7,2018-04-06
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,20,2018-04-03
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,4,2018-04-04
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,17,2018-04-08


### Outliers

In [97]:
# From the data below, the max values of Bluecar_counter, Charge_slots and slots seems to be far 
# from the rest of the data. Futhermore they're far from the 75th percentile. further investigation 
# should be done to see if they affect the data.
# Values for Postal_code and hours do not count in this, as they are uniquely standardized units.

Bluecar.describe()

,Bluecar_counter,Charge_Slots,Postal_code,Slots,hour
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,2.333600,0.253400,82634.878400,1.932400,11.509200
std,2.035274,0.546304,8835.865721,1.905402,6.893549
min,0.000000,0.000000,75001.000000,0.000000,0.000000
25%,0.000000,0.000000,75012.000000,0.000000,6.000000
50%,2.000000,0.000000,75019.000000,1.000000,11.000000
75%,4.000000,0.000000,92320.000000,3.000000,18.000000
max,7.000000,2.000000,95880.000000,7.000000,23.000000


In [98]:
# From the data below, the max value seems to be far from the rest of the data It's far 
# from the 75th percentile, further investigation should be done to see if it affects the data 
Bluecar.hour.value_counts()
# This gives us a hint that 21:00 hrs could be the most popular hour. this will be investigated
# during data analysis.

21    237
9     221
16    220
7     219
6     219
3     218
2     218
13    213
8     211
5     211
11    210
18    210
19    208
14    207
20    202
23    201
12    200
15    200
4     199
17    198
22    196
1     194
10    194
0     194
Name: hour, dtype: int64

In [99]:
# First we'll store a copy of the original in order to manipulate the data and still have a copy of
# the original to compare results with

Bluecar_with_possible_outliers = Bluecar.copy()
Bluecar_with_possible_outliers.head()

,Address,Bluecar_counter,Charge_Slots,Charging_Status,City,Postal_code,Rental_status,Scheduled_at,Slots,Station_type,Status,hour,Date
0,2 Avenue de Suffren,0,0,nonexistent,Paris,75015,operational,NaN,2,station,ok,11,2018-04-08
1,145 Rue Raymond Losserand,6,0,operational,Paris,75014,operational,NaN,0,station,ok,7,2018-04-06
2,2 Avenue John Fitzgerald Kennedy,3,0,operational,Le Bourget,93350,operational,NaN,1,station,ok,20,2018-04-03
3,51 Rue EugÃ¨ne OudinÃ©,3,1,operational,Paris,75013,operational,NaN,2,station,ok,4,2018-04-04
4,6 avenue de la Porte de Champerret,3,0,nonexistent,Paris,75017,operational,NaN,3,station,ok,17,2018-04-08
